In [1]:
#Embedding generation using emb_dataloader which checks the path and verifies the file type and images

In [1]:
# some packages need async support in jupyter
import nest_asyncio
nest_asyncio.apply()

In [2]:
# layer_name can also be given 
# for name, module in model.named_modules():
#     if isinstance(module, torch.nn.AdaptiveAvgPool2d):  # Look for avgpool layers
#         print(name, module)
# output:
# avgpool AdaptiveAvgPool2d(output_size=(1, 1))
# layer4.2.downsample.1 AdaptiveAvgPool2d(output_size=(1, 1))

## layer_name = 'self.model.layer4[2].downsample[1]' or layer='avgpool'

In [3]:
from mb_pytorch.models.modelloader import ModelLoader
from mb_pytorch.utils.generate_emb import EmbeddingGenerator
from mb_utils.src.logging import logger

In [4]:
ModelLoader?

Init signature: ModelLoader(data: dict, logger=None)
Docstring:     
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as regular attributes::

    import torch.nn as nn
    import torch.nn.functional as F

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 20, 5)
            self.conv2 = nn.Conv2d(20, 20, 5)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            return F.relu(self.conv2(x))

Submodules assigned in this way will be registered, and will have their
parameters converted too when you call :meth:`to`, etc.

.. note::
    As per the example above, an ``__init__()`` call to the parent class
    must be made before assignment on the child.

:ivar training: Boolean represents whether this module is in training or
                

In [5]:
EmbeddingGenerator?

Init signature:
EmbeddingGenerator(
    model,
    dataloader,
    layer='avgpool',
    layer_name=None,
    device='cpu',
    logger=None,
) -> None
Docstring:      <no docstring>
Init docstring:
Initialize the EmbeddingGenerator class
Input:
    model: model for which embeddings are to be generated
    dataloader: dataloader for the data
    layer: layer from which embeddings are to be generated. Default: avgpool. if None, then it will look for the layer_name
    layer_name: name of the layer. Only used if layer is None. Default: None. Eg: 'self.model.layer4[2].downsample[1]'
    device: device on which the model is to be run
    logger: logger instance
File:           ~/.local/lib/python3.11/site-packages/mb_pytorch/utils/generate_emb.py
Type:           type
Subclasses:     